# 1) Análisis de comportamiento de los usuarios de Megaline.

En este análisis vamos a explorar el comportamiento de los usuarios de la compañía Megaline.
Lo que buscamos lograr es conocer cuales usuarios están migrando de plan de servicio.
Queremos que menos usuarios usen planes heredados y mejor migren a uno de los dos planes contemporáneos: Smart o Ultra.
Vamos a usar la data disponible para estudiar el comportamiento del usuario y poder ofrecer al perfil adecuado el plan ideal según su comportamiento.
En eset caso estamos hablando de un problemas de clasificación de data, así que usaremos modelos de clasificación para este análisis.

In [1]:
import pandas as pd

df = pd.read_csv('users_behavior.csv') #Leer base de datos

print(df.info()) #Revisar información disponible
print()

df['calls'] = df['calls'].astype('int64') #Cambiamos el tipo de dato a los datos correctos para trabajar con ellos
df['messages'] = df['messages'].astype('int64')

print(df.info()) #Comprobamos el cambio
print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB
None



In [2]:
print(f"El total de duplicados es de: {df.duplicated().sum()}") #Revisamos duplicados
print()
print(df.isna().sum()) #Revisamos datos nulos


El total de duplicados es de: 0

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


# 2) Segmentación de la información para entrenar el modelo

Ahora que revisamos la información disponible y limpiamos la base, ya la podemos usar para el análisis.
En esta siguiente seccion vamos a dividir la data para el entrenamiento y prueba.
También en este paso crearemos un dataset de validación, características y objetivo de nuestro modelo.

In [3]:
pip install scikit-learn #Instalar librería para poder usar funciones de machine learning

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sklearn
from sklearn.model_selection import train_test_split #Importar fórmula para dividir la data.

In [8]:
train_data, test_data  = train_test_split(df, test_size=0.20, random_state=12345) #Dividimos la data em dos para entrenar el modelo. Usamos un modelo 80/20 para la compración de datos. 80%  de entrenamiento, 20% para laa pruebas.
train_data_val, validation_data = train_test_split(train_data, test_size=0.25) #Dividimos la data de entrenamiento para poder tener un data set de validación del modelo entrenado.

features_train = train_data_val.drop(['is_ultra'], axis=1) #Indicamos las características a entrenar
value_train = train_data_val['is_ultra'] #Indicamos el objetivo que buscamos con el modelo

features_test = test_data.drop(['is_ultra'], axis=1) #Indicamos las características a probar
value_test = test_data['is_ultra'] #Indicamos las características a probar

features_val = validation_data.drop(['is_ultra'], axis=1) #Indicamos características de validación
value_val = validation_data['is_ultra'] #Indicamos objetivos de validación


# 3) Crear modelos de predicción usando modelos de decisiones para clasificación.

Ya que tenemos la data lista para ser usada, ahora vamos a usar diferentes modelos de predicción para la clasificación de los planes.
En este segmento buscamos validar cual es el mejore modelo de clasificación que debmos usar para este ejercicio.
Usaremos un árbol de decisión y un bosque de decisión para realizar las clasifiaciones-
Aunque vayamos a usar modelos de clasificación, vale la pena usar un modelo de regresión logística para los modelos.
Este modelo de regrsión también nos puede indicar a que usuarios serían más viable recomendar el plan que más se asemeje a su tipo de consumo.

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
tree_model = DecisionTreeClassifier(random_state=12345)
tree_model.fit(features_train, value_train)
print("Entrenamiento: ", tree_model.score(features_train, value_train)) #Aqui validamos la exactitud del bosque
print("Validación: ", tree_model.score(features_val, value_val))
print("Prueba: ", tree_model.score(features_test, value_test)) #Usar un árbol decisión para este problema no es eficiente, necesitamos un modelo más robusto

Entrenamiento:  1.0
Validación:  0.7278382581648523
Prueba:  0.687402799377916


In [16]:
from sklearn.ensemble import RandomForestClassifier #Importamos modelo para el bosque de decisión

In [31]:
for trees in range (1,31): 
    forest_model = RandomForestClassifier(n_estimators=trees, random_state=12345) #Almacenamos nuestro modelo e indicamos un estado de azar, también definimos la cantidad de arboles dentro de nuestro bosque
    forest_model.fit(features_train, value_train) #Creamos nuestro modelo entrenado para Bosque de Decisiones
    print("Cantidad de árboles:", trees)
    print("Entrenamiento: ", forest_model.score(features_train, value_train)) #Aqui validamos la exactitud del bosque
    print("Validación: ", forest_model.score(features_val, value_val))
    print("Prueba: ", forest_model.score(features_test, value_test))
    print()
    #Creamos un for para poder conocer cuántos árboles me van a brindar la validación más alta de mi modelo.
    #En este caso tener 28 árboles dentro del modelo nos dá la validación más alta

Cantidad de árboles: 1
Entrenamiento:  0.9087136929460581
Validación:  0.7371695178849145
Prueba:  0.7060653188180405

Cantidad de árboles: 2
Entrenamiento:  0.9056016597510373
Validación:  0.7698289269051322
Prueba:  0.7325038880248833

Cantidad de árboles: 3
Entrenamiento:  0.9470954356846473
Validación:  0.7651632970451011
Prueba:  0.749611197511664

Cantidad de árboles: 4
Entrenamiento:  0.946058091286307
Validación:  0.7713841368584758
Prueba:  0.7620528771384136

Cantidad de árboles: 5
Entrenamiento:  0.9683609958506224
Validación:  0.7838258164852255
Prueba:  0.7527216174183515

Cantidad de árboles: 6
Entrenamiento:  0.9642116182572614
Validación:  0.7884914463452566
Prueba:  0.7744945567651633

Cantidad de árboles: 7
Entrenamiento:  0.9761410788381742
Validación:  0.8009331259720062
Prueba:  0.776049766718507

Cantidad de árboles: 8
Entrenamiento:  0.9735477178423236
Validación:  0.7884914463452566
Prueba:  0.7776049766718507

Cantidad de árboles: 9
Entrenamiento:  0.9839211618

In [32]:
forest_model = RandomForestClassifier(n_estimators=28, random_state=12345) 
forest_model.fit(features_train, value_train) 
print("Cantidad de árboles:", 28)
print("Entrenamiento: ", forest_model.score(features_train, value_train))
print("Validación: ", forest_model.score(features_val, value_val))
print("Prueba: ", forest_model.score(features_test, value_test))

Cantidad de árboles: 28
Entrenamiento:  0.9953319502074689
Validación:  0.8102643856920684
Prueba:  0.7916018662519441


In [10]:
from sklearn.linear_model import LogisticRegression #Importamos los modelos que usaremos para realizar las predicciones

In [37]:
linear_model = LogisticRegression(random_state=12345) #Almacenamos nuestro modelo para ser usado e indicamos un estado de azar
linear_model.fit(features_train, value_train) #Creamos nuestro modelo de entrenamiento para Regresión Linear
print("Entrenamiento: ", linear_model.score(features_train, value_train)) #Aqui validamos la exactitud del modelo linear
print("Validación: ", linear_model.score(features_val, value_val)) 
print("Prueba: ", linear_model.score(features_test, value_test)) #Aunque este modelo se acerca a nuestro objetivo, la validación es menor al 0.75 que buscamos.
#Será mejor tomar algún modelo anterior para realizar la predicción. Aunque este modelo sea fácil de utilizar y ahorre recursos, en este caso no es el ideal.

Entrenamiento:  0.7551867219917012
Validación:  0.7309486780715396
Prueba:  0.7589424572317263


# 4) Conclusiones

En este caso lo mejor sería un bosque de decisición para la clasificación de los datos.
Este modelo es más robusto y cumple con nuestro estándar de exactitud para la predicción.
Este modelo usa varios árboles de decisiones y compara los resultados para llegar a la mejor exactitud.
